In [ ]:
import pandas as pd
import json
from collections import defaultdict
import firebase_admin
from firebase_admin import credentials, auth
from firebase_admin import db, firestore
import datetime
import numpy as np


<h1>Firebase
 

In [ ]:
cred = credentials.Certificate(
    r"D:\Workstuff\my-work-python-script\Database_in_pandas\Firebase_Shipment_Database\json_secret\logistec-firebase-adminsdk-mz58b-63b513fce5.json")
firebase_admin.initialize_app(cred, {})

firestore_db = firestore.client()


<h1>UTIL_49

In [ ]:
# read clipboard
data = pd.read_clipboard(sep='\t')

# create new dataframe with selected columns
stockout_id = data["ID"].astype(str).str.replace(",", "")
branchid = data["Branch (ID)"].astype(str).str.replace(",", "")
branchname = data["Branch (Name)"]
phyid = pd.DataFrame(stockout_id + "-" +
                     branchid).rename(columns={0: "PHYID"}, inplace=False)
docdate = pd.to_datetime(
    data["Doc Date"], dayfirst=True).dt.strftime("%d/%m/%y")
productcode = data["Product (Code)"].astype(str)
productname = data["Product (Name)"]
totalnumber = data["Number"]
cat_name = data["Category Name"]
serial = data["Serial"].fillna(data.pop('Product (Code)')).astype(str)

frames = [docdate, branchname, phyid, productcode,
          productname, serial, totalnumber, cat_name]
new_frames = pd.concat(frames, axis=1)

# grouping every columns again to sum all duplicates + set new index.
grouped = new_frames.groupby(['Doc Date', 'Branch (Name)', 'PHYID', 'Product (Code)', 'Product (Name)', 'Serial', 'Category Name'],
                             as_index=False, dropna=False).agg({'Number': 'sum'}).set_index(["PHYID", 'Product (Code)', 'Serial'], drop=False)
grouped_phyidonly = new_frames["PHYID"].unique()
grouped_serialonly = new_frames["Serial"].unique()

# convert into dictionary (maps) for firestore


def nested_dict(): return defaultdict(dict)
def nested_list(): return defaultdict(list)


nested = nested_list()
nested_header = nested_dict()
nested_product = nested_dict()
nested2 = nested_dict()
product_list = []

total_phyid = grouped["PHYID"].nunique()
total_items = sum(grouped["Number"])

nested2 = {'Total_Bill': total_phyid, 'Total_Items': total_items}
# create some kind of metadata first

#

for nested_row in grouped.itertuples():
    nested[nested_row.Index[0]].append({'Product Code': str(nested_row.Index[1]), 'Product Name': nested_row._5, 'Serial': str(
        nested_row.Index[2]), 'Number': nested_row.Number, 'Category Name': nested_row._7})
    nested_header[nested_row.Index[0]] = {
        'PHYID': nested_row.Index[0], 'Doc Date': nested_row._1, 'Branch': nested_row._2, }
    nested_product[nested_row.Index[0]][str(nested_row.Index[2])] = {'Product Code': str(
        nested_row.Index[1]), 'Product Name': nested_row._5, 'Serial': str(nested_row.Index[2]), 'Number': nested_row.Number, 'Category Name': nested_row._7}

new_nested = {}
product_nested = {}
for key, value in nested_header.items():
    if key in nested:
        new_nested[key] = {
            'PHYID': value['PHYID'],
            'Doc Date': value['Doc Date'],
            'Branch': value['Branch'],
            'Products': nested[key],
        }
        product_nested[key] = nested_product[key]


In [ ]:
product_nested

<H1>SEND DATA

In [ ]:


# send to firestore
print(len(nested))
today = datetime.datetime.today()
batch = firestore_db.batch()
transaction = firestore_db.transaction()

#batch_size = 500
# write bill data
for number, field in enumerate(grouped_phyidonly):
    if today.day >= 15:
        return49_data = firestore_db.collection("Return49_Data").document("Year").collection(
            str(today.year)).document("Month").collection(str(today.month)).document(field)
    else:
        return49_data = firestore_db.collection("Return49_Data").document("Year").collection(
            str(today.year)).document("Month").collection(str(today.month - 1)).document(field)
    if number % 500 == 499:
        batch.commit()
        batch = firestore_db.batch()
    batch.set(return49_data, new_nested[field], merge=True)
batch.commit()


for number, field in enumerate(grouped_phyidonly):
    if today.day >= 15:
        return49Product = firestore_db.collection("Return49_Data").document("Year").collection(str(today.year)).document(
            "Month").collection(str(today.month)).document(field).collection("Products").document(field)
    else:
        return49Product = firestore_db.collection("Return49_Data").document("Year").collection(str(today.year)).document(
            "Month").collection(str(today.month - 1)).document(field).collection("Products").document(field)
    if number % 500 == 300:
        batch.commit()
        batch = firestore_db.batch()
    batch.set(return49Product, product_nested[field], merge=True)
batch.commit()

# for number, field in enumerate(grouped_phyidonly):
# if today.day >= 15:
##            return49_data = firestore_db.collection("Return49_Data").document("Year").collection(str(today.year)).document("Month").collection(str(today.month)).document(field).collection("Products")
# else:
##            return49_data = firestore_db.collection("Return49_Data").document("Year").collection(str(today.year)).document("Month").collection(str(today.month - 1)).document(field).collection("Products")

# write total data
if today.day >= 15:
    return49_data_util = firestore_db.collection("Return49_Data_Util").document("Year").collection(
        str(today.year)).document("Month").collection(str(today.month)).document("Total_return49_data")
else:
    return49_data_util = firestore_db.collection("Return49_Data_Util").document("Year").collection(
        str(today.year)).document("Month").collection(str(today.month - 1)).document("Total_return49_data")


@firestore.transactional
def update_in_transaction(transaction, return49_data_util):
    snapshot = return49_data_util.get(transaction=transaction)
    if snapshot.exists:
        transaction.update(return49_data_util, {'Total_Bill': snapshot.get(
            'Total_Bill') + nested2['Total_Bill'], 'Total_Items': snapshot.get('Total_Items') + nested2['Total_Items']})
    else:
        transaction.set(return49_data_util, {
                        'Total_Bill': nested2['Total_Bill'], 'Total_Items': nested2['Total_Items']}, merge=True)


update_in_transaction(transaction, return49_data_util)


In [ ]:
with pd.ExcelWriter("test.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace",) as excel:
    grouped.to_excel(excel, sheet_name="test")


In [ ]:
nested2['Total Bill']
